In [1]:
import os
import re
import spacy
import whisper
import torch
import subprocess

 
device = "cuda" if torch.cuda.is_available() else "cpu"
 
model = whisper.load_model("base", device=device)

def download_audio(youtube_url, output_path="audio.mp3"):
    """Download the audio from a YouTube video."""
    try:
        command = [
            "yt-dlp",
            "--format", "bestaudio",
            "--extract-audio",
            "--audio-format", "mp3",
            "--output", output_path,
            youtube_url,
        ]
        subprocess.run(command, check=True)
        print(f"Audio downloaded to {output_path}")
        return output_path
    except Exception as e:
        print(f"Error downloading audio: {e}")
        return None

def transcribe_audio(audio_path):
    """Transcribe the audio using Whisper."""
    try:
        print("Extracting transcript")
        result = model.transcribe(audio_path)
        return result  # Contains text and timestamps
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None

def save_transcript_with_timestamps(transcript, output_file="transcript.txt"):
    """Save the transcript with timestamps to a file."""
    try:
        with open(output_file, "w") as file:
            for segment in transcript['segments']:
                start_time = segment['start']
                end_time = segment['end']
                text = segment['text']
                file.write(f"[{start_time:.2f} - {end_time:.2f}] {text}\n")
        print(f"Transcript saved to {output_file}")
    except Exception as e:
        print(f"Error saving transcript: {e}")


 

def detect_terms_to_exclude(text:str):
    """Detect terms that are likely to be technical or domain-specific."""
  
    nlp = spacy.load("en_core_web_trf")

    patterns = [
        {
            "label": "TECH",
            "pattern": [
                {"lower": "machine"},
                {"lower": "learning"}
            ]
        },
        {
            "label": "TECH",
            "pattern": [
                {"lower": "artificial"},
                {"lower": "intelligence"}
            ]
        },
        {
            "label": "TECH",
            "pattern": [
                {"lower": "data"},
                {"lower": "science"}
            ]
        },
        {
            "label": "TECH",
            "pattern": [
                {"TEXT": "ML"},
            ]
        },
          {
            "label": "PERSON",
            "pattern": [
                {"TEXT": "Rover"},
            ]
        },
        {
            "label": "TECH",
            "pattern": [{"TEXT": {"REGEX": "(?i)^ai$"}}]
        },
        {
                "label": "LINK",
                "pattern": [
                    {"TEXT": {"REGEX": r"https?://[^\s]+"}}
                ]
        }
    ]

    ruler = nlp.add_pipe("entity_ruler", before="ner") 
    ruler.add_patterns(patterns)

    doc = nlp(text)     
 
    exclude_terms = []
 
    for ent in doc.ents:
        exclude_terms.append(ent.text)
    
 
    for token in doc:
        if token.pos_ == "PROPN" or (token.is_upper and len(token) > 1):  
            exclude_terms.append(token.text)

    exclude_terms = [term for term in exclude_terms if not re.search(r"(\n|%|\d{1,2}%|^\d{1,2}$)", term)]
    exclude_terms = list(set(exclude_terms))
    
    return exclude_terms

 
 
def generate_translation_prompt(english_text, exclude_terms, custom_terms):
 
    exclusion_list = "\n".join([f"- {term}" for term in exclude_terms])
 
    custom_translations = "\n".join([f"- \"{key}\" should be translated as \"{value}\"" for key, value in custom_terms.items()])
    
  
    instructions = f"""
    Translate the following English text to Burmese (Myanmar) while ensuring the following:
    
    1. **Exclude specific terms from translation**:
        {exclusion_list}
        These terms should remain in English and not be translated.
    
    2. **For specific phrases, use custom translations**:
        {custom_translations}
        These phrases should be translated exactly as indicated.

    The translation should respect the context and accurately convey the meaning of the original text while maintaining these exclusions and custom translations.
    """
 
    prompt = f"{instructions}\n\nHere is the English text you need to translate:\n\n{english_text}"

    return prompt


def save_file(filepath: str, content: str):
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)
        
def read_file(filepath:str):
    with open(filepath , "r") as f:
        return f.read()


c:\Users\heinm\OneDrive\Desktop\mm-tts\.venv\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_locatio

In [2]:

from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer


def translate_english_to_myanmar_with_transformer(text, exclude_terms=None, custom_terms=None):
    if exclude_terms is None:
        exclude_terms = []
    if custom_terms is None:
        custom_terms = {}
 
    placeholders = {}
    for idx, (eng_term, myanmar_term) in enumerate(custom_terms.items()):
        placeholder = f"__CUSTOM_TERM_{idx}__"
        text = text.replace(eng_term, placeholder)
        placeholders[placeholder] = myanmar_term

    
    for idx, term in enumerate(exclude_terms):
        placeholder = f"__TERM_{idx}__"
        text = text.replace(term, placeholder)
        placeholders[placeholder] = term
    print(f"optimzed text: {text}")
    model_name = "facebook/m2m100_418M"
    tokenizer = M2M100Tokenizer.from_pretrained(model_name)
    model = M2M100ForConditionalGeneration.from_pretrained(model_name)
    
    tokenizer.src_lang = "en"
    encoded_text = tokenizer(text, return_tensors="pt")

    
    generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.get_lang_id("my"))
    translated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

     
    for placeholder, term in placeholders.items():
        translated_text = translated_text.replace(placeholder, term)

    return translated_text
 


def translate_with_cohere(text, exclude_terms=None, custom_terms=None):
    import cohere
    api_key=""
    co = cohere.Client(api_key=api_key)
    if exclude_terms is None:
        exclude_terms = []
    if custom_terms is None:
        custom_terms = {}
 
    exclusion_text = "\n".join([f"- {term}" for term in exclude_terms])
    custom_terms_text = "\n".join([f"'{eng_term}' => '{myanmar_term}'" for eng_term, myanmar_term in custom_terms.items()])

 
    prompt = f"""
    Translate the following English text to Burmese (Myanmar) while keeping the original English terms for special terms.
    Please exclude the following terms from translation: 
    {exclusion_text}
    Also, translate the following custom terms as specified:
    {custom_terms_text}

    English text:
    {text}
    """

   
    response = co.generate(
        model="command-r-08-2024", 
        prompt=prompt,
        max_tokens=300,
        temperature=0.7
    )

   
    translated_text = response.generations[0].text

    return translated_text



In [ ]:
youtube_url = "https://www.youtube.com/watch?v=DmgGGUYn2c8"
output_filename = "trnascript.txt"
audio_file = download_audio(youtube_url)

if audio_file:

    transcript = transcribe_audio(audio_file)
    
    if transcript:

        save_transcript_with_timestamps(transcript, output_filename)

    os.remove(audio_file)

In [3]:
 
transcript_filepath = "transcripts/transcript.txt"

english_text = read_file(transcript_filepath)


my_exclude_terms = ["Abilities", "character", "icon", "players", "player", "play", "damage", "normal attack", "action", "actions"]

exclude_terms = detect_terms_to_exclude(english_text)
exclude_terms = exclude_terms + my_exclude_terms


print("Terms to Exclude:", exclude_terms)
# old
#Terms to Exclude: ['Quick', 'Concerto', 'Swap', 'DPS', 'Menu', 'Dodge & Counters', 'Ultimates / Liberation', 'Ultimate', 'Havoc', 'Circuit', 'Perfect', 'Echo', 'Parries', 'Echoes', 'itâ€', 'Energy', 'Utilities', 'Normal, Heavy, and Plunge Attacks', 'Counters', 'One', 'the Utilities Menu', 'Every Forte Circuit', 'Counter', 'Intro', 'Attacks', 'Dodge', 'Normal', 'Ultimates', 'Stagger', 'Gadget', 'Sprinting', 'one', 'charactersâ€', 'though', 'Skill', 'Skills', 'their Intro Skill', 'Damage', 'Heavy', 'the Forte Circuit', 'The Forte Circuit', 'Concerto Energy / Outro', 'Liberation', 'Bars', 'Thoughts', 'Gadgets', 'master', 'Grapple', 'Outro', 'Forte Circuit', 'Plunge', 'Stamina', 'Closing', 'Concerto Energy', 'Ult', 'Intro Skill', 'Forte', 'Bar', 'the Concerto Energy bar', 'Abilities', 'character', 'icon', 'players', 'player', 'play', 'damage', 'normal attack', 'action', 'actions']

c:\Users\heinm\OneDrive\Desktop\mm-tts\.venv\Lib\site-packages\thinc\shims\pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(fil

Terms to Exclude: ['Skill', 'Skills', 'Menu', 'Liberation', 'Bar', 'Forte Circuit', 'Ultimates', 'Abilities', 'Stagger', 'Ultimate', 'one', 'Quick', 'Swapping', 'Plunge', 'Outro Skills', 'Bars', 'Concerto Energy', 'the Forte Circuit', 'an Outro Skill', 'Perfect', 'Ult', 'One', 'Ultimates / Liberation', 'Gadget', 'Havoc', 'Stamina', 'Energy', 'Dodge', 'Outro', 'Grapple', 'DPS', 'Concerto', 'Damage', 'Concerto Energy / Outro /', 'Intro', 'Echoes', 'Forte', 'Echo', 'Circuit', 'Abilities', 'character', 'icon', 'players', 'player', 'play', 'damage', 'normal attack', 'action', 'actions']


In [7]:
print(" ".join(exclude_terms))

Quick Concerto Swap DPS Menu Dodge & Counters Ultimates / Liberation Ultimate Havoc Circuit Perfect Echo Parries Echoes itâ€ Energy Utilities Normal, Heavy, and Plunge Attacks Counters One the Utilities Menu Every Forte Circuit Counter Intro Attacks Dodge Normal Ultimates Stagger Gadget Sprinting one charactersâ€ though Skill Skills their Intro Skill Damage Heavy the Forte Circuit The Forte Circuit Concerto Energy / Outro Liberation Bars Thoughts Gadgets master Grapple Outro Forte Circuit Plunge Stamina Closing Concerto Energy Ult Intro Skill Forte Bar the Concerto Energy bar Abilities character icon players player play damage normal attack action actions


In [11]:
custom_terms = {
        "for each character": "character တစ်ကောင်ချင်းစီအတွက်",
    }

transcript_filepath = "transcripts/transcript.txt"

original_timestamp_text = read_file(transcript_filepath)
myanmar_text = generate_translation_prompt(original_timestamp_text, exclude_terms=exclude_terms, custom_terms=custom_terms)


In [12]:
# Example usage
save_file("prompt.txt", myanmar_text)


In [ ]:
pattern = r"\[\d+\.\d{2} - \d+\.\d{2}\]"
 
 
text = read_file("transcripts/transcript.txt") 
timestamps = re.findall(pattern, text)

time_stamps ="" 
time_stamps += "\n".join(timestamps)

save_file("timestamps.txt", time_stamps)


In [ ]:
import torch
import numpy as np
import re
import uuid
from transformers import pipeline
import soundfile as sf

if torch.cuda.is_available():
    device = "cuda"

myanmar_model_name = "facebook/mms-tts-mya"
english_model_name = "facebook/mms-tts-eng"
myanmar_model = pipeline("text-to-speech", model=myanmar_model_name)
english_model = pipeline("text-to-speech", model=english_model_name)

sample_rate = 16000
pause_duration = 0.5  

myanmar_text = """
ကဲ ဒါဆိုရင်တော့, character ရဲ့ Abilities တွေကို စတင်ကြည့်ရှုရအောင်
Ult, Echo, Skill, Gadget, Forte Circuit, နဲ့ Concerto Energy.
ဒါတွေက character တစ်ကောင်ချင်းစီအတွက်
အဓိက Abilities တွေဖြစ်ပါတယ်။
ပုံကဝိုင်း icon လေးဟာ  lock-on feature ဖြစ်ပါတယ်။
"""

def generate_audio(text, model):
    speech = model(text)
    audio_data = speech["audio"]
    if audio_data.ndim == 2 and audio_data.shape[0] == 1:
        audio_data = audio_data.squeeze(0)
    return audio_data


# Extract English words and Myanmar text segments
segments = re.split(r'(\b[A-Za-z]+\b)', myanmar_text)
audio_segments = []

for segment in segments:
    segment = segment.strip()
    if not segment:
        continue
    if re.match(r'^[A-Za-z]+$', segment):
     
        print(f"segment (English): {segment}")
        audio_segments.append(generate_audio(segment, english_model))
    else:
        if segment.strip() in [",", ".", "။", "၊", ":", ";","-"]:
            continue
      
        print(f"transcript (Myanmar): {segment}")
        audio_segments.append(generate_audio(segment, myanmar_model))
    
    # Add pause for newline characters
    if '\n' in segment:
        pause = np.zeros(int(pause_duration * sample_rate)) 
        audio_segments.append(pause)

# Combine all audio segments
combined_audio = np.concatenate(audio_segments)
 
random_uuid = uuid.uuid4()
output_file = f"samples/combined_audio_{random_uuid}.wav"
sf.write(output_file, combined_audio, sample_rate)
print(f"Audio saved as {output_file}")

Device set to use cuda:0


OSError: facebook/mms-tts-en is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

FileNotFoundError: [WinError 2] The system cannot find the file specified